In [ ]:
try:
    import google.colab
    on_colab = True

    !gdown --id 11xL65BOrEc93TFYhCp7VmrEh2MWBNgwf -O depth.zip
    !unzip depth.zip -d ./depth

    !gdown --id 1l5xwvj3YZgKIDWSoE8Geom_FT9rSbKpn -O depth_edge.zip
    !unzip depth_edge.zip -d ./depth_edges

    from google.colab import  drive
    drive.mount('/content/drive')

except ImportError:
    on_colab = False

In [ ]:
from google.colab import  drive
drive.mount('/content/drive')

In [9]:
!rm -r depth_pattern_analysis/

In [ ]:
if on_colab:
    !git clone https://github.com/hamidrezafahimi/depth_pattern_analysis.git

In [1]:
on_colab = False

In [ ]:
!pip install opencv-python

In [7]:
import os
import sys

# Directories for depth images and template masks.
if on_colab:
    sys.path.append('/content/depth_pattern_analysis/lib')
    depth_dir = "/content/depth"
    tmask_dir = "/content/depth_edges"
    output_dir = "/content/drive/MyDrive/outs5"
else:
    sys.path.append("../../lib")
    depth_dir = "../../data/depth"
    tmask_dir = "../../data/depth_edge"
    output_dir = "../../data/outs"

# Get sorted list of filenames.
depth_files = sorted([f for f in os.listdir(depth_dir) if os.path.isfile(os.path.join(depth_dir, f))])
tmask_files = sorted([f for f in os.listdir(tmask_dir) if os.path.isfile(os.path.join(tmask_dir, f))])

# Check that both directories have exactly the same filenames.
if set(depth_files) != set(tmask_files):
    raise ValueError("Depth images and template masks must have matching filenames.")

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

from searchBasedDepthDiffuser import SearchBasedDepthDiffuser
# Create a ThresholdSearcher object with desired parameters and logging flags.
searcher = SearchBasedDepthDiffuser(outdir=output_dir)

In [ ]:
import cv2
import time
# Process each matching pair.
for fname in depth_files:
    depth_path = os.path.join(depth_dir, fname)
    tmask_path = os.path.join(tmask_dir, fname)

    depth_image = cv2.imread(depth_path, cv2.IMREAD_GRAYSCALE)
    if depth_image is None:
        raise IOError(f"Could not load depth image: {depth_path}")
    template_mask = cv2.imread(tmask_path, cv2.IMREAD_GRAYSCALE)
    if template_mask is None:
        raise IOError(f"Could not load template mask: {tmask_path}")
    if depth_image.shape != template_mask.shape:
        raise ValueError(f"Image shape mismatch for {fname}")

    t0 = time.time()
    searcher.diffuse(template_mask, depth_image, fname[:-4])
    print("time: ", time.time() - t0)
    print("-------------")